Try the lubin FPdock code on nluc and a selection of SmBit sequences. Run a good number of decoys to get an idea if this will be similar to the server.

In [1]:
# my attempt to adapt the above code.

from pyrosetta import *
from pyrosetta.rosetta.core.select.residue_selector import \
	ChainSelector, OrResidueSelector, ResidueIndexSelector
from pyrosetta.rosetta.protocols.constraint_generator import \
	AddConstraints, CoordinateConstraintGenerator
from pyrosetta.rosetta.protocols.enzdes import ADD_NEW, AddOrRemoveMatchCsts
from pyrosetta.rosetta.protocols.flexpep_docking import FlexPepDockingProtocol
#import design_protease as dp


def fp_dock_init(sf='ref2015'):
    #opts = '-run:preserve_header -mute core'
    #opts += ' -pep_refine -ex1 -ex2 -use_input_sc -flip_HNQ -no_optH false -score:weights ' + sf
    #init(opts)
    init()

    

def pep_run(decoy_name, n_decoys, pep_sequence=None, starts_at=None,  pdb='nluc_native_test.pdb', sf='ref2015'):
    
    # Score function and starting PDB
    #sf = create_score_function(sf) # no idea what this sf is...
    pose = pose_from_pdb(pdb)
    # Changing peptide sequence
    #asyn_seq = "GVTGWRLCERILA"
    #asyn_seq = "SVTGWRLCERILA"
    #starts_at = 158
    
    # pasted from mut_scan_purge.ipynb
    # 3. create ScoreFuncions for the Interface and "ddG" calculations
    # the pose's Energies objects MUST be updated for the Interface object to
    #    work normally
    sf = get_fa_scorefxn() #  create_score_function('standard')
    sf(pose)    # needed for proper Interface calculation
    # end paste
    
    if pep_sequence:
        pose = dp.make_residue_changes(pose, sf, pep_sequence, starts_at, None, None)

    # Creating FlexPepDock protocol using init options
    fpdock = FlexPepDockingProtocol()

    #decoy_name = 'test1'
    jd = PyJobDistributor(decoy_name, n_decoys, sf)
    while not jd.job_complete:
        pp = Pose()
        pp.assign(pose)
        fpdock.apply(pp)
        jd.output_decoy(pp)

Previously:
```python
from multiprocessing import Pool

sequences = [
    "GVTGWRLCERILA",
    "NVSGWRLFKKISN",
    "GVTGYRLFEEILG" # ideally the glycines on each end would not be there. need to figure out how to delete residues.
    
]

with Pool() as p:
    work = [(seq, 158, "./decoys/"+"test2_"+seq, 3, "nluc_native_test.pdb", 'ref2015_cst') for seq in sequences]
    p.starmap(pep_run, work)
```

In [2]:
from os import listdir
pdb_files = listdir("./smbits/")
[f[:-4] for f in pdb_files]

['peptide79',
 'peptide104',
 'native_test',
 'peptide86',
 'peptide114',
 'peptide128',
 'peptide99',
 'peptide101',
 'peptide78']

In [3]:
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
print(timestr)

20210320-104424


In [ ]:
from multiprocessing import Pool

fp_dock_init()
timestr = time.strftime("%Y%m%d-%H%M%S")

with Pool() as p:
    work1 = [("./decoys/"+filename[:-4]+"-"+timestr+"_1", 50, None, None, "./smbits/"+filename, 'ref2015') for filename in pdb_files]
    work2 = [("./decoys/"+filename[:-4]+"-"+timestr+"_2", 50, None, None, "./smbits/"+filename, 'ref2015') for filename in pdb_files]
    p.starmap(pep_run, work1+work2)

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.CentOS.python37.Release 2020.10+release.46415fa3e9decb8b6e91a4e065c15543eb27a461 2020-03-05T09:09:24] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=410136626 seed_offset=0 real_seed=410136626 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=410136626 RG_type=mt19937
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: {0} Database f